# Gensim介绍

* 从原始的非结构化的文本中，无监督地学习到文本隐层的主题向量表达。
* 它支持包括TF-IDF，LSA，LDA，和word2vec在内的多种主题模型算法，
* 支持流式训练，并提供了诸如相似度计算，信息检索等一些常用任务的API接口
* https://zhuanlan.zhihu.com/p/37175253

## 基本概念

* 语料（Corpus）：一组原始文本的集合，用于无监督地训练文本主题的隐层结构。语料中不需要人工标注的附加信息。在Gensim中，Corpus通常是一个可迭代的对象（比如列表）。每一次迭代返回一个可用于表达文本对象的稀疏向量。
* 向量（Vector）：由一组文本特征构成的列表。是一段文本在Gensim中的内部表达。
* 稀疏向量（SparseVector）：通常，我们可以略去向量中多余的0元素。此时，向量中的每一个元素是一个(key, value)的元组
* 模型（Model）：是一个抽象的术语。定义了两个向量空间的变换（即从文本的一种向量表达变换为另一种向量表达）。

## 训练预料预处理

将原始文本转换成Gensim可以处理的稀疏向量

In [48]:
# 词袋模型 corpus的每一个元素对应一篇文档。text就是输入的语料库
texts = [['human', 'interface', 'computer'],
['survey', 'user', 'computer', 'system', 'response', 'time'],
['eps', 'user', 'interface', 'system'],
['system', 'human', 'system', 'eps'],
['user', 'response', 'time'],
['trees'],
['graph', 'trees'],
['graph', 'minors', 'trees'],
['graph', 'minors', 'survey']]
print(texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [49]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [52]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0]) # [(0, 1), (1, 1), (2, 1)]
print(corpus[1]) # (3，1) 这个元素代表第二篇文档中id为3的单词“user”出现了1次。
print(corpus[2]) # 按照id大小进行排列，不是原文中的顺序
print(corpus[3]) # （5，2）代表id为5的单词在改文档中出现了两次

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]


* 得到了语料中每一篇文档对应的稀疏向量（这里是bow向量）
* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples



中文例子

In [24]:
import jieba
from gensim import corpora, models, similarities
raw_documents = [  
    '0无偿居间介绍买卖毒品的行为应如何定性',  
    '1吸毒男动态持有大量毒品的行为该如何认定',  
    '2如何区分是非法种植毒品原植物罪还是非法制造毒品罪',  
    '3为毒贩贩卖毒品提供帮助构成贩卖毒品罪',  
    '4将自己吸食的毒品原价转让给朋友吸食的行为该如何认定',  
    '5为获报酬帮人购买毒品的行为该如何认定',  
    '6毒贩出狱后再次够买毒品途中被抓的行为认定',  
    '7虚夸毒品功效劝人吸食毒品的行为该如何认定',  
    '8妻子下落不明丈夫又与他人登记结婚是否为无效婚姻',  
    '9一方未签字办理的结婚登记是否有效',  
    '10夫妻双方1990年按农村习俗举办婚礼没有结婚证 一方可否起诉离婚',  
    '11结婚前对方父母出资购买的住房写我们二人的名字有效吗',  
    '12身份证被别人冒用无法登记结婚怎么办？',  
    '13同居后又与他人登记结婚是否构成重婚罪',  
    '14未办登记只举办结婚仪式可起诉离婚吗',  
    '15同居多年未办理结婚登记，是否可以向法院起诉要求离婚'  
] 
print(raw_documents)

['0无偿居间介绍买卖毒品的行为应如何定性', '1吸毒男动态持有大量毒品的行为该如何认定', '2如何区分是非法种植毒品原植物罪还是非法制造毒品罪', '3为毒贩贩卖毒品提供帮助构成贩卖毒品罪', '4将自己吸食的毒品原价转让给朋友吸食的行为该如何认定', '5为获报酬帮人购买毒品的行为该如何认定', '6毒贩出狱后再次够买毒品途中被抓的行为认定', '7虚夸毒品功效劝人吸食毒品的行为该如何认定', '8妻子下落不明丈夫又与他人登记结婚是否为无效婚姻', '9一方未签字办理的结婚登记是否有效', '10夫妻双方1990年按农村习俗举办婚礼没有结婚证 一方可否起诉离婚', '11结婚前对方父母出资购买的住房写我们二人的名字有效吗', '12身份证被别人冒用无法登记结婚怎么办？', '13同居后又与他人登记结婚是否构成重婚罪', '14未办登记只举办结婚仪式可起诉离婚吗', '15同居多年未办理结婚登记，是否可以向法院起诉要求离婚']


In [26]:
texts = [[word for word in jieba.cut(document, cut_all=True)] for document in raw_documents]
print(texts)

[['0', '无偿', '居间', '介绍', '买卖', '毒品', '的', '行为', '应', '如何', '定性'], ['1', '吸毒', '男', '动态', '持有', '大量', '毒品', '的', '行为', '该', '如何', '认定'], ['2', '如何', '区分', '是非', '非法', '种植', '毒品', '原', '植物', '罪', '还是', '是非', '非法', '法制', '制造', '毒品', '罪'], ['3', '为', '毒贩', '贩卖', '贩卖毒品', '毒品', '提供', '帮助', '构成', '贩卖', '贩卖毒品', '毒品', '罪'], ['4', '将', '自己', '吸食', '的', '毒品', '原价', '转让', '让给', '朋友', '吸食', '的', '行为', '该', '如何', '认定'], ['5', '为', '获', '报酬', '帮', '人', '购买', '毒品', '的', '行为', '该', '如何', '认定'], ['6', '毒贩', '出狱', '后', '再次', '够买', '毒品', '途中', '被', '抓', '的', '行为', '认定'], ['7', '虚夸', '毒品', '功效', '劝', '人', '吸食', '吸食毒品', '毒品', '的', '行为', '该', '如何', '认定'], ['8', '妻子', '下落', '下落不明', '不明', '丈夫', '又', '与', '他人', '登记', '结婚', '是否', '为', '无效', '婚姻'], ['9', '一方', '未', '签字', '办理', '的', '结婚', '结婚登记', '登记', '是否', '有效'], ['10', '夫妻', '双方', '1990', '年', '按', '农村', '习俗', '举办', '婚礼', '没有', '结婚', '结婚证', '', '', '一方', '方可', '可否', '起诉', '离婚'], ['11', '结婚', '婚前', '对方', '父母', '出资', '购买', '的', '住房', '写', '我们', '二人', '的', '名字', '

In [28]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(141 unique tokens: ['0', '买卖', '介绍', '如何', '定性']...)


In [32]:
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[2])

[(3, 1), (8, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 2)]


## 主题向量的变换

在Gensim中，每一个向量变换的操作都对应着一个主题模型，例如上一小节提到的对应着词袋模型的doc2bow变换。每一个模型又都是一个标准的Python对象。下面以TF-IDF模型为例，介绍Gensim模型的一般使用方法。

In [36]:
from gensim import models
tfidf = models.TfidfModel(corpus)
print(tfidf)

TfidfModel(num_docs=16, num_nnz=214)


In [37]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # [(0, 0.70710678), (1, 0.70710678)]

[(0, 0.7071067811865475), (1, 0.7071067811865475)]


## 文档相似度的计算

在得到每一篇文档对应的主题向量后，我们就可以计算文档之间的相似度，进而完成如文本聚类、信息检索之类的任务。在Gensim中，也提供了这一类任务的API接口。

以信息检索为例。对于一篇待检索的query，我们的目标是从文本集合中检索出主题相似度最高的文档。

首先，我们需要将待检索的query和文本放在同一个向量空间里进行表达（以LSI向量空间为例）：

In [40]:
# 构造LSI模型并将待检索的query和文本转化为LSI主题向量
# 转换之前的corpus和query均是BOW向量
query = corpus[2:4] 
lsi_model = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
documents = lsi_model[corpus]
query_vec = lsi_model[query]

In [45]:
index = similarities.MatrixSimilarity(documents)
sims = index[query_vec] #返回一个元组类型的迭代器：(idx, sim)
print(sims) 

[[ 0.9920135   0.99315065  1.0000001   0.9960572   0.9900997   0.9896967
   0.98062474  0.9949924  -0.02910087  0.19547951 -0.15231332  0.69516414
  -0.11633774 -0.04324021 -0.11418902 -0.13650814]
 [ 0.9992917   0.9996002   0.9960572   1.          0.9986482   0.9984964
   0.99413687  0.99993634  0.05968961  0.2817106  -0.06403454  0.7561949
  -0.02776812  0.04556062 -0.02560575 -0.04808705]]
